In [10]:
if (!require("tidyverse")) { install.packages("tidyverse")
  require("tidyverse") };
library("tidyverse");

if (!require("reshape")) { install.packages("reshape")
  require("reshape") };
library("reshape");

if (!require("data.table")) { install.packages("data.table")
  require("data.table") };
library("data.table");

In [11]:
MyData <- read.csv(file="BNC2_sample.csv", header=FALSE, sep=",");
colnames(MyData)<-c("Code","Date","Open","High","Low","Close","Volume","VWAP","TWAP")
head(MyData)

Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
GWA_BTC,2014-04-01,467.2762,488.6226,467.2762,479.5625,74776.48,482.7574,482.8155
GWA_BTC,2014-04-02,479.1967,494.2979,431.3180,437.0787,114052.96,460.1924,465.9325
GWA_BTC,2014-04-03,437.3265,449.7381,414.4062,445.5970,91415.08,432.2859,433.2828
GWA_BTC,2014-04-04,445.1796,456.1005,429.1615,449.8113,51147.27,443.4581,443.9253
GWA_BTC,2014-04-05,450.0802,464.0881,445.1607,461.6951,28449.19,452.5258,452.9498
GWA_BTC,2014-04-06,463.1802,469.1714,451.1295,461.0219,21720.64,460.9213,460.8858


In [12]:
NewData <- MyData %>%
filter(str_detect(Code, "GWA_"))

In [13]:
PivotedData <- cast(NewData, Date ~ Code, value = "VWAP")
PivotedData <- PivotedData[order(as.Date(PivotedData$Date, format="%m/%d/%Y")), ]
tail(PivotedData)

,Date,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
1389,2018-01-18,11792.87,1059.9565,193.1625,0.5123853,1.683890
1390,2018-01-19,11826.36,1068.4513,194.9981,0.5114904,1.820644
1391,2018-01-20,13062.68,1158.7099,207.5807,0.5223238,1.752069
1392,2018-01-21,12326.23,1108.9048,197.3641,0.4828454,1.551756
1393,2018-01-22,11397.52,1038.2056,184.9183,0.4703197,1.426731
1394,2018-01-23,10921.00,992.0523,176.9474,0.4730853,1.422020


In [14]:
rownames(PivotedData) <- PivotedData$Date
PivotedData$Date <- NULL

In [15]:
Shifted <- lapply(PivotedData,lag,n=7)
Rate_of_change <-  PivotedData/Shifted - 1
tail(Rate_of_change)

Rate_of_change$Date <- rownames(Rate_of_change)

,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
2018-01-18,-0.1859704,-0.1758900,-0.1925633,-0.04259439,-0.21838609
2018-01-19,-0.1787354,-0.1709744,-0.1808487,-0.21322572,-0.21581759
2018-01-20,-0.1306220,-0.1867120,-0.1817972,-0.22812837,-0.29016826
2018-01-21,-0.1452760,-0.2010230,-0.2159841,-0.21953378,-0.29502853
2018-01-22,-0.2143725,-0.2413735,-0.2405269,-0.25215016,-0.32272051
2018-01-23,-0.1138674,-0.1226110,-0.1293543,-0.02151248,-0.04223546


In [20]:
MeltedData <- melt(Rate_of_change, id="Date")
colnames(MeltedData)[3] <- "delta_7"
#write.csv(Rate_of_change, file = "Rate_of_change.csv")
head(MeltedData,10)

Date,variable,delta_7
2014-04-01,GWA_BTC,NA
2014-04-02,GWA_BTC,NA
2014-04-03,GWA_BTC,NA
2014-04-04,GWA_BTC,NA
2014-04-05,GWA_BTC,NA
2014-04-06,GWA_BTC,NA
2014-04-07,GWA_BTC,NA
2014-04-08,GWA_BTC,-0.05964323
2014-04-09,GWA_BTC,-0.02121777
2014-04-10,GWA_BTC,-0.07043877


In [17]:
delta_dict <- list(7,14,21,28)
names(delta_dict) <- list("delta_7","delta_14","delta_21","delta_28")

melted_dfs <- list()

for (i in delta_dict)
{
  transit <- lapply(PivotedData,lag,n=i)
  rate_change <-  PivotedData/transit - 1
  rate_change$Date <- rownames(rate_change)
  melted_dfs[[paste("delta_",i)]] <- melt(rate_change, id="Date")
  colnames(melted_dfs[[paste("delta_",i)]])[2] <- "Code"
  colnames(melted_dfs[[paste("delta_",i)]])[3] <- paste("delta_",i)
}

In [18]:
transit <- lapply(PivotedData,lead,n=7)
rate_change <-  PivotedData/transit - 1
rate_change$Date <- rownames(rate_change)
melted_dfs[["return_7"]] <- melt(rate_change, id="Date")
colnames(melted_dfs[["return_7"]])[2] <- "Code"
colnames(melted_dfs[["return_7"]])[3] <- "return_7"

Merged_Return <- Reduce(function(x,y) merge(x,y,by=c("Date","Code"),all=TRUE),melted_dfs)

base_Data = select(NewData,'Date', 'Code', 'Volume', 'VWAP')



In [19]:
Merged_Data <- merge(base_Data,Merged_Return,by=c("Date","Code"),all=TRUE)
head(Merged_Data,10)

Date,Code,Volume,VWAP,delta_ 7,delta_ 14,delta_ 21,delta_ 28,return_7
2014-04-01,GWA_BTC,74776.48,482.75743985,NA,NA,NA,NA,0.06342617
2014-04-01,GWA_ETH,NA,NA,NA,NA,NA,NA,NA
2014-04-01,GWA_LTC,713104.55,13.19192122,NA,NA,NA,NA,0.15756377
2014-04-01,GWA_XLM,NA,NA,NA,NA,NA,NA,NA
2014-04-01,GWA_XRP,3342635.94,0.00868482,NA,NA,NA,NA,0.09202909
2014-04-02,GWA_BTC,114052.96,460.19242000,NA,NA,NA,NA,0.02167773
2014-04-02,GWA_ETH,NA,NA,NA,NA,NA,NA,NA
2014-04-02,GWA_LTC,1479611.86,12.09030356,NA,NA,NA,NA,0.06495046
2014-04-02,GWA_XLM,NA,NA,NA,NA,NA,NA,NA
2014-04-02,GWA_XRP,7321246.47,0.00772967,NA,NA,NA,NA,-0.05153932
